In [1]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup

In [2]:
def lyrics(artist, title):
    base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query='
    url = f'{base_url}{quote(artist+title)}+%EA%B0%80%EC%82%AC'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    lyr = soup.select_one('.lyrics_txt._lyrics_txt').get_text().strip()

    return lyr

In [3]:
import re
import joblib
new_cvect = joblib.load('model/imdb_cvect_2.pkl')
new_lrc = joblib.load('model/imdb_lrc2.pkl')

In [4]:
text = lyrics('에스파', 'life too short')
text = re.sub('[^A-Za-z]', ' ', text)
review_cv = new_cvect.transform([text])
'긍정' if new_lrc.predict(review_cv)[0] == 1 else '부정'

'긍정'

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
a = cvect.fit_transform([text]).toarray()[0]
print(a)
print(cvect.vocabulary_)

[1 2 1 1 6 1 1 4 9 2 1 6 1 1 1 1 3 1 1 6 3 6 4 2 3 1 1 3 1 2 1 2 2 1 3 1 1
 2 2 3 3 2 3 1 7 1 1 2 2 1 6 1 1 6 1 1 1 2 1 1 1 2 1 3 1 1 1 3 1 1 1]
{'discussiontalking': 6, 'nothingyou': 33, 'say': 44, 'faceyou': 10, 'think': 56, 'words': 70, 'gospelbut': 15, 'troublei': 58, 'ain': 0, 'got': 16, 'time': 57, 'wasteyou': 64, 'need': 31, 'lifecause': 23, 'life': 22, 'shortyou': 48, 'bored': 1, 'mindyou': 29, 'really': 38, 'nonsensesomewhere': 32, 'elsecause': 9, 've': 61, 'realizei': 37, 'doing': 7, 'regardlessand': 40, 'don': 8, 'carewhat': 4, 'itand': 21, 'matterif': 27, 'like': 24, 'noti': 34, 'having': 19, 'fun': 11, 'sowhy': 50, 'stopdoing': 53, 'regardlessno': 42, 'imma': 20, 'waythat': 67, 'wanti': 63, 'stopsome': 55, 'people': 35, 'meanall': 28, 'phone': 36, 'screenwhen': 45, 'tryna': 59, 'live': 25, 'liveswhy': 26, 'gotta': 17, 'viciousbe': 62, 'business': 2, 'stead': 52, 'getting': 12, 'mineyou': 30, 'regardlessdoing': 41, 'regardlesswhy': 43, 'stopoh': 54, 'won': 69, 'turn': 60, '

In [6]:
for i in range(len(a)):
    i == cvect.vocabulary_.values()

In [10]:
# 필요한 라이브러리 가져오기
from gensim.summarization import summarize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NLTK 패키지의 불용어 다운로드
nltk.download('stopwords')
nltk.download('punkt')

# 불용어 설정
stop_words = set(stopwords.words('english')) # 사용 언어에 맞게 변경해야 함

# 가사 요약 함수 정의
def summarize_lyrics(lyrics):
    # 텍스트 전처리
    tokens = word_tokenize(lyrics.lower()) # 토큰화
    tokens = [token for token in tokens if token.isalpha()] # 알파벳 문자만 포함
    tokens = [token for token in tokens if token not in stop_words] # 불용어 제거

    # 요약
    summary = summarize(' '.join(tokens))
    
    return summary

# 가사 입력 받기
input_lyrics = '''A hopeless romantic all my life
Surrounded by couples all the time
I guess I should take it as a sign
(oh why oh why, oh why oh why)

I’m feeling lonely (lonely)
Oh I wish I’d find a lover
that could hold me (hold me)
Now I’m crying in my room
So skeptical of love
(say what you say but I want it more)
But still I want it more, more, more

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

I look for his arrows everyday
I guess he got lost or flew away
Waiting around is a waste (waste)
Been counting the days since November
Is loving as good as they say?

Now I’m so lonely (lonely)
Oh I wish I’d find a lover
that could hold me (hold me)
Now I’m crying in my room
So skeptical of love
(say what you say but I want it more)
But still I want it more, more, more

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

Hopeless girl is seeking
someone who will share this feeling
I’m a fool, a fool for love, a fool for love

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb'''

# 가사 요약 출력
summary = summarize_lyrics(input_lyrics)
print("가사 요약:")
print(summary)

ModuleNotFoundError: No module named 'gensim.summarization'

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 문장 토큰화
    sentences = sent_tokenize(text)
    # 단어 토큰화
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    # 불용어 제거
    words = [word for word in words if word not in stop_words]
    return words, sentences

def calculate_sentence_scores(words, sentences):
    tokenizer = RegexpTokenizer(r'\w+')
    # 단어의 빈도수 계산
    word_freq = FreqDist(words)
    # 문장의 점수 계산
    scores = []
    for sentence in sentences:
        sentence_words = tokenizer.tokenize(sentence)
        sentence_score = sum([word_freq[word] for word in sentence_words])
        scores.append(sentence_score)
    return scores

def generate_summary(text):
    words, sentences = preprocess_text(text)
    scores = calculate_sentence_scores(words, sentences)
    # 문장 점수 정규화
    max_score = np.max(scores)
    normalized_scores = [score / max_score for score in scores]
    # 문장 추출
    summary = ""
    for sentence, score in zip(sentences, normalized_scores):
        if score > 0.5:
            summary += sentence + " "
    return summary

# 가사 입력 받기
input_lyrics = '''A hopeless romantic all my life
Surrounded by couples all the time
I guess I should take it as a sign
(oh why oh why, oh why oh why)

I’m feeling lonely (lonely)
Oh I wish I’d find a lover
that could hold me (hold me)
Now I’m crying in my room
So skeptical of love
(say what you say but I want it more)
But still I want it more, more, more

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

I look for his arrows everyday
I guess he got lost or flew away
Waiting around is a waste (waste)
Been counting the days since November
Is loving as good as they say?

Now I’m so lonely (lonely)
Oh I wish I’d find a lover
that could hold me (hold me)
Now I’m crying in my room
So skeptical of love
(say what you say but I want it more)
But still I want it more, more, more

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

Hopeless girl is seeking
someone who will share this feeling
I’m a fool, a fool for love, a fool for love

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb

I gave a second chance to Cupid
But now I’m left here feeling stupid
Oh the way he makes me feel
that love isn’t real
Cupid is so dumb'''

# 가사 요약 출력
summary = generate_summary(input_lyrics)
print("가사 요약:")
print(summary)


가사 요약:
a hopeless romantic all my life
surrounded by couples all the time
i guess i should take it as a sign
(oh why oh why, oh why oh why)

i’m feeling lonely (lonely)
oh i wish i’d find a lover
that could hold me (hold me)
now i’m crying in my room
so skeptical of love
(say what you say but i want it more)
but still i want it more, more, more

i gave a second chance to cupid
but now i’m left here feeling stupid
oh the way he makes me feel
that love isn’t real
cupid is so dumb

i look for his arrows everyday
i guess he got lost or flew away
waiting around is a waste (waste)
been counting the days since november
is loving as good as they say? now i’m so lonely (lonely)
oh i wish i’d find a lover
that could hold me (hold me)
now i’m crying in my room
so skeptical of love
(say what you say but i want it more)
but still i want it more, more, more

i gave a second chance to cupid
but now i’m left here feeling stupid
oh the way he makes me feel
that love isn’t real
cupid is so dumb

hopeles

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YONSAI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YONSAI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.utils import get_stop_words

def summarize_text(text, num_sentences=1):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summarizer.stop_words = get_stop_words("english")
    summary = summarizer(parser.document, num_sentences)
    
    result = []
    for sentence in summary:
        result.append(str(sentence))
    
    return " ".join(result)

# 사용 예시
input_text = '''Uh, uh
Too spicy
You want my A to the Z
But you won't 어림없지
맞혀봐 sweet 또는 freak
What's hiding underneath? (I see)
망설이듯 간 보는 너 기회는 없어 oh
No, you won't get it
Nah, nah, you won't get it, I say (hey)
깜빡 한순간 끌어당겨 you'll be mine (mine)
살짝 다가와 can cross my borderline (hey)
(Line, line, line) 널 따분하게 했던 everyday (day)
흥미로운 덫을 던져줄게
뛰어들어 봐 just right now
'Cause I'm too spicy for your heart
Ring the fire alarm
심장을 파고들어 넌 I'm too spicy
번지는 자극 속에 넌
바로 그 순간
또 다른 나를 발견해
I'm too spicy, too, too, I'm too spicy
Don't stop 겁내지 마 (hey)
Bang, bang 외쳐봐
I'm too spicy, yeah, I'm too spicy, you know that I
Don't stop 용기 내 봐 (hey)
Next step, myself
I'm too spicy, too spicy, too, too, I'm too spicy
Tell me what you see (see)
When you look at me (me)
'Cause I am a ten out of ten honestly
기세가 다른 move (move) 널 압도하는 groove (groove)
But you keep wasting your time, dudе
Don't chase me 경고해 난
Erase mе 멀리 달아나
Hey 이젠 알겠니?
원래부터 bad, I'm so bad honestly, so bad
Hey 전부 가질래 지금부턴 call you mine (mine)
도망칠 거면 don't cross my borderline (hey)
(Line, line, line) 늘 뻔하기만 했던 everyday (day)
이제 그만 rule을 벗어날 때
어렵지 않아 just right now
'Cause I'm too spicy for your heart
Ring the fire alarm
심장을 파고들어 넌 I'm too spicy
번지는 자극 속에 넌
바로 그 순간
또 다른 나를 발견해
I'm too spicy, too, too, I'm too spicy
Ready? uh
좀 더 강도를 높여 다음, 다음, 다음 (hoo)
Ayy, ayy, ayy, one of a kind
Ooh-hoo 우린 한계를 앞서 wow, wow, wow
That's right, oh, oh, oh, oh
'Cause I'm too spicy for your heart
(Oh) ring the fire alarm
새로운 도전 끝에 넌 (ooh) I'm too spicy
변화할 시간이야 넌
지금 이 순간 (so hot) 또 다른 나를 찾아내
I'm too spicy, too, too, I'm too spicy
Don't stop 겁내지 마 (hey)
Bang, bang 외쳐봐
I'm too spicy, yeah, I'm too spicy, you know that I
Don't stop 용기 내 봐 (hey)
Next step, myself
I'm too spicy, too spicy, too, too, I'm too spicy
'Cause I, oh, yeah
Yeah, I'm too spicy for your heart
(Yeah, I'm too spicy for your heart)
마침내 번져오는 joy (enjoy, joy, joy, joy)
Yeah, I'm too spicy for your heart
I'm too spicy, too, too, I'm too spicy for you'''

summary = summarize_text(input_text)
print(summary)


원래부터 bad, I'm so bad honestly, so bad Hey 전부 가질래 지금부턴 call you mine (mine) 도망칠 거면 don't cross my borderline (hey) (Line, line, line) 늘 뻔하기만 했던 everyday (day) 이제 그만 rule을 벗어날 때 어렵지 않아 just right now 'Cause I'm too spicy for your heart Ring the fire alarm 심장을 파고들어 넌 I'm too spicy 번지는 자극 속에 넌 바로 그 순간 또 다른 나를 발견해 I'm too spicy, too, too, I'm too spicy Ready?
